##  The CHSH Game


_course: quantum cryptography for beginners
<br>date: 10 october 2022
<br>author: burton rosenberg_


The CHSH game has a strategy that uses an entangled pair of qubits to win a game of chance with great probability than can be achieved with any non-quantum strategy. The essence of the game is two plays flip coins and then post an outcome, we will call it red or green, that is scored according to the coin flips.

If the players could communicate after the coin flips, their winnings would be perfect. But they cannot. Neither knows the others coin flips, hence must play the odds as best as they can. Essentially the players must predict when the coin flips were heads-heads. They can only win at this game one out of four trials.

However, if each player has one qubit of an entangled pair of qubits, they can make a measurement on their quibit that depends on the coin flip. The entanglement will correlate the measures so that the players can signal when they have tossed a heads, allowing for a coordinated action in the case of heads-heads.

The experiment is named after John Clauser, Michael Horne, Abner Shimony, and Richard Holt, and was proposed in 1969. It was based on trying to prove Bell's Inequality, which gave a framework to test certain hidden-variable theories. 


### The game

Following the tradition in describing cryptographic protocols, our two players are Alice and Bob.

Alice and Bob each have a machine with a button that when pushed flips a fair coin. The box has two lights on top, one red and one green, the Alice and Bob will light to signal their decision based on the coin flip. They also half each one half of an EPR pair.

To assure Alice and Bob do not communicate after the coin flips, the Alice and Bob and their boxes will be placed light years apart, and their buttons will be pushed simultaneously by using a laser signal sent from a midpoint location between Alice and Bob. The light colors and a message about the coin flips will be sent back to the umpire at the source location of the laser, to determine if the game has been won or list.

<pre>

      (R)  (G)                       (R)  (G)
       |    |                         |    |
    +----------+                   +----------+
    |   [0/1]  ===|  &lt;-- * --&gt;  |===   [0/1]  |
    |          |                   |          |
    |   |&phi;&gt;    |                   |   |&phi;&gt;    |
    +----------+                   +----------+
       Alice                           Bob
</pre>

The umpire, on receiving from Alice and Bob their coin flips, and seeing the color of their lights, scores this way,

| Alice's coin | Bob's coin | Lights |
|:-:|:-:|:- |
| 0 | 0 | same color |
| 1 | 0 | same color |
| 0 | 1 | same color |
| 1 | 1 | different colors |

Hence Alice and Bob both have a strategies $\cal{A}(a,h), \cal{B}(b,h) \in\{\,\!\mbox{R, G}\}$
with $a,b\in_R \{\,0,1\,\}$, two unbaised independently drawn random variables, and $h$ some common strategy data.

### Classical computation


It was noted that classically, strategies exist to win 3/4 of the time. 


#### Null strategy

One strategy is for Alice and Bob to always play green. This makes they always correct in all cases except heads-heads, and in that case they are always wrong. Hence the probabilty of winning is the probabily of avoiding the case heads-heads, or 3/4.

#### Mixed Randomized and Null

Alice and Bob can trade-off certainty in all cases for some winning in all cases by having Alice always play green and Bob play green on tails, but on heads play green with probability $p$ and red with probability $1-p$. Working through the math, this still gives 3/4 for all $p$. With $p=0$ we have the null strategy, and with $p=1$ we have another deterministic strategy, this one with tails-heads always losing and the other three cases always winning.


#### Optimal strategy

An argument that 3/4 is the optimal win amount can be based on a reduction argument, in which:

- From a strategy to win with greater than 3/4 can be create a algorithm to predict a fair coin with success greater than 1/2.

It seems a valid axiom that we cannot do such a prediction, and therefore we cannot have strategy with a win greater than 3/4.

#### Quantum

However, if Alice and Bob can communicate, they can simply reveal their coins and will all the time. The CHSH game is constructed to make classical communication between Alice and Bob impossible, but uses an entangled qubit pair to create a channel of correlation between Alice and Bob. 


### Quantum computation with entanglement.

Alice and Bob share a Bell state, $|00\rangle+|11\rangle$. Alice will measure one of the qubits in a basis depending on her coin flip, and Bob will do so on the other qubit. The measurement basis will be described by $|\phi\rangle$ described by the Bloch angle $\theta$ (the measurements will be in the $x$-$z$ plane). 

| Alice's coin | Alice's $\theta$ | Bob's coin | Bob's $\theta$  |
|:-:|:-:|:-:|:-:|
| 0 | 0 | 0 | $\pi/4$ |
| 1 | $\pi/2$ | 1 | $-\pi/4$ |

That is, Alice will measure on the Z basis for tails and the X basis for heads. Bob's measurements do not have standard terminolgy, but all these measurements are,

$$
M_\theta = \pmatrix{\cos \theta/2 & \sin \theta/2 \\ \sin \theta/2 &  -\cos \theta/2 \\ }
$$

We run the experiments. Looking at the historgram of results, we see that for 3 of the 4 coin flips, we decidedly have a positive correlation between the Alice and Bob's measurements; for the fourth coin flip result, we decidely have a negative correlation. This is all we need to the fashion out the narrative of red or green lights.

### Measuring at &pi;/8

In all of the cases favorable to a win, the two measurements are being made at $\pi/8$ from each other. Hence the advantage over all will be,

$$
\cos^2 \pi/8 = \left({{1}\over{2}}\,\sqrt{2+\sqrt{2}}\right)^2 = 0.8536
$$

In [1]:
import numpy as np
import math

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi


def alice_bob_circuit(a,b):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    if a==1:
        qc.h(0)
    if b==0:
        qc.ry(-math.pi/4.0,1)
    else:
        qc.ry(math.pi/4.0,1)
    qc.measure_all()
    return qc

def alice_bob_measurements(simulator,verbose=True):
    counts = {}
    for a in range(2):
        for b in range(2):
            qc = alice_bob_circuit(a,b)
            if verbose:
                print(f'\nAlice: {a}, Bob: {b}')
                print(qc.draw(output='text'))
            qc = transpile(qc, simulator)
            result = simulator.run(qc).result()
            counts[(a,b)] = result.get_counts(qc)
            if verbose:
                print(counts[a,b])
    return counts

# Transpile for simulator
simulator = Aer.get_backend('aer_simulator')
counts = alice_bob_measurements(simulator)



Alice: 0, Bob: 0
        ┌───┐                  ░ ┌─┐   
   q_0: ┤ H ├──■───────────────░─┤M├───
        └───┘┌─┴─┐┌──────────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(-π/4) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 
{'01': 82, '11': 425, '10': 73, '00': 444}

Alice: 0, Bob: 1
        ┌───┐                 ░ ┌─┐   
   q_0: ┤ H ├──■──────────────░─┤M├───
        └───┘┌─┴─┐┌─────────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/4) ├─░──╫─┤M├
             └───┘└─────────┘ ░  ║ └╥┘
meas: 2/═════════════════════════╩══╩═
                                 0  1 
{'01': 63, '10': 89, '11': 433, '00': 439}

Alice: 1, Bob: 0
        ┌───┐        ┌───┐     ░ ┌─┐   
   q_0: ┤ H ├──■─────┤ H ├─────░─┤M├───
        └───┘┌─┴─┐┌──┴───┴───┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(-π/4) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 
{'01': 71, '00': 433, '10':

In [2]:
# We now go ahead and assign red and green to the measurement 
# outcomes (said as 0 and 1), and score the game.

def scoring(coins,counts):
        win = lose = 0 
        if coins[0]*coins[1]==1:
            # head-heads
            for k in counts:
                # win on different colors
                if k=='10' or k=='01':
                    win += counts[k]
                else:
                    lose += counts[k]
        else:
            # not head-heads
            for k in counts:
                # win on same colors
                if k=='00' or k=='11':
                    win += counts[k]
                else:
                    lose += counts[k]
        return(win,lose)
    
def alice_bob_scoring(counts):
    
    def make_into_colors(k):
        color = {'0':'G','1':'R'}
        return ''.join([color[k[0]],'',color[k[1]]])

    overall = 0.0
    for a in range(2):
        for b in range(2):
            print(f'\nAlice: {a}, Bob: {b}')
            c = counts[(a,b)]
            score = scoring((a,b),c)
            for k in c:
                print(f'\t{make_into_colors(k)}: {c[k]}')
            print(f'\twin: {score[0]}, lose: {score[1]}')
            w = score[0]/(score[0]+score[1])
            overall += w 
            print(f'\tadvantage: {w:.4f}')
    return overall/4.0
                
w = alice_bob_scoring(counts)
print(f'\nGame advantage: {w:.4f}')



Alice: 0, Bob: 0
	GR: 82
	RR: 425
	RG: 73
	GG: 444
	win: 869, lose: 155
	advantage: 0.8486

Alice: 0, Bob: 1
	GR: 63
	RG: 89
	RR: 433
	GG: 439
	win: 872, lose: 152
	advantage: 0.8516

Alice: 1, Bob: 0
	GR: 71
	GG: 433
	RG: 81
	RR: 439
	win: 872, lose: 152
	advantage: 0.8516

Alice: 1, Bob: 1
	GG: 77
	RR: 72
	RG: 438
	GR: 437
	win: 875, lose: 149
	advantage: 0.8545

Game advantage: 0.8516


### Exercise A

The Bell pair might seem to only allow for final states $|00\rangle$ or $|11\rangle$, as suggested by how it is written. However, whatever the measurement is made on one qubit, the other qubit will assume that state.

Devise an experiement to demonstrate this.



In [3]:
# your answer here


    

### Exercise B

The Bell pair must be kept entangled else the game will not work completely. Devise an experiment to learn how disurbing the entanglement will affect the game.

_Hint:_ Insert a measurement on one of the qubits before the measurement and see how that changes the advantage of the game.


In [4]:
# your experiement here



### Exercise C

Explore the choice of Bob's measurement. Change Bob's measurement from $\pi/8$ to values closer to 0 and see how that effects the advantage of the game.

_Hint:_ Redfine alice_bob_circuit() then use all the other code.

In [5]:

# your code here



#### END

## Answers

### Repeating the preliminary code.


In [6]:
# repeat the preliminaries
# repeating the code from above


import numpy as np
import math

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi

simulator = Aer.get_backend('aer_simulator')

def alice_bob_measurements(simulator,verbose=True):
    counts = {}
    for a in range(2):
        for b in range(2):
            qc = alice_bob_circuit(a,b)
            if verbose:
                print(f'\nAlice: {a}, Bob: {b}')
                print(qc.draw(output='text'))
            qc = transpile(qc, simulator)
            result = simulator.run(qc).result()
            counts[(a,b)] = result.get_counts(qc)
            if verbose:
                print(counts[a,b])
    return counts

def scoring(coins,counts):
        win = lose = 0 
        if coins[0]*coins[1]==1:
            # head-heads
            for k in counts:
                # win on different colors
                if k=='10' or k=='01':
                    win += counts[k]
                else:
                    lose += counts[k]
        else:
            # not head-heads
            for k in counts:
                # win on same colors
                if k=='00' or k=='11':
                    win += counts[k]
                else:
                    lose += counts[k]
        return(win,lose)
    
def alice_bob_scoring(counts):
    
    def make_into_colors(k):
        color = {'0':'G','1':'R'}
        return ''.join([color[k[0]],'',color[k[1]]])

    overall = 0.0
    for a in range(2):
        for b in range(2):
            print(f'\nAlice: {a}, Bob: {b}')
            c = counts[(a,b)]
            score = scoring((a,b),c)
            for k in c:
                print(f'\t{make_into_colors(k)}: {c[k]}')
            print(f'\twin: {score[0]}, lose: {score[1]}')
            w = score[0]/(score[0]+score[1])
            overall += w 
            print(f'\tadvantage: {w:.4f}')
    return overall/4.0
 

### Exercise A: A solution

In [7]:
# exercise A
# the experiement is to use the RY gate to rotate the measurement 
# to various angles, and check that the result is still either 00 or 11 (entangled)

def create_bell_pair():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    return qc

def measure_at_theta(qc,theta):
    qc.ry(theta,0)
    qc.ry(theta,1)
    qc.measure_all()
    return qc

for i in range(13):
    theta = math.pi/12*i
    qc = measure_at_theta(create_bell_pair(),theta)
    print(qc.draw(output='text'))
    qc = transpile(qc, simulator)
    result = simulator.run(qc).result()
    print(result.get_counts(qc))
    

        ┌───┐     ┌───────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(0) ├─░─┤M├───
        └───┘┌─┴─┐├───────┤ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(0) ├─░──╫─┤M├
             └───┘└───────┘ ░  ║ └╥┘
meas: 2/═══════════════════════╩══╩═
                               0  1 
{'00': 501, '11': 523}
        ┌───┐     ┌──────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(π/12) ├─░─┤M├───
        └───┘┌─┴─┐├──────────┤ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/12) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 
{'00': 515, '11': 509}
        ┌───┐     ┌─────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(π/6) ├─░─┤M├───
        └───┘┌─┴─┐├─────────┤ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/6) ├─░──╫─┤M├
             └───┘└─────────┘ ░  ║ └╥┘
meas: 2/═════════════════════════╩══╩═
                                 0  1 
{'00': 527, '11': 497}
        ┌───┐     ┌─────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(π/4) ├─░─┤M├───
        └───┘┌─┴─┐├─────────┤ ░ └╥┘┌─┐
  

### Exercise B: A solution


In [8]:
# overwriting the old circuit with a new 
# one, that measures one of the qubits

# my solution is to place a measurement ahead of the strategy measurements, 
# and see the lack of coherence for some of the coin flips, and its impact
# on the game adavantage

def alice_bob_circuit(a,b):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    qc.measure([1],[0])
    if a==1:
        qc.h(0)
    if b==0:
        qc.ry(-math.pi/4.0,1)
    else:
        qc.ry(math.pi/4.0,1)
    qc.measure([0,1],[0,1])
    return qc

simulator = Aer.get_backend('aer_simulator')
counts = alice_bob_measurements(simulator)
w = alice_bob_scoring(counts)
print(f'\nGame advantage: {w:.4f}')


Alice: 0, Bob: 0
     ┌───┐                    ┌─┐   
q_0: ┤ H ├──■─────────────────┤M├───
     └───┘┌─┴─┐┌─┐┌──────────┐└╥┘┌─┐
q_1: ─────┤ X ├┤M├┤ Ry(-π/4) ├─╫─┤M├
          └───┘└╥┘└──────────┘ ║ └╥┘
c: 2/═══════════╩══════════════╩══╩═
                0              0  1 
{'01': 71, '10': 67, '00': 438, '11': 448}

Alice: 0, Bob: 1
     ┌───┐                   ┌─┐   
q_0: ┤ H ├──■────────────────┤M├───
     └───┘┌─┴─┐┌─┐┌─────────┐└╥┘┌─┐
q_1: ─────┤ X ├┤M├┤ Ry(π/4) ├─╫─┤M├
          └───┘└╥┘└─────────┘ ║ └╥┘
c: 2/═══════════╩═════════════╩══╩═
                0             0  1 
{'01': 61, '10': 85, '00': 435, '11': 443}

Alice: 1, Bob: 0
     ┌───┐     ┌───┐            ┌─┐   
q_0: ┤ H ├──■──┤ H ├────────────┤M├───
     └───┘┌─┴─┐└┬─┬┘┌──────────┐└╥┘┌─┐
q_1: ─────┤ X ├─┤M├─┤ Ry(-π/4) ├─╫─┤M├
          └───┘ └╥┘ └──────────┘ ║ └╥┘
c: 2/════════════╩═══════════════╩══╩═
                 0               0  1 
{'01': 264, '00': 239, '10': 265, '11': 256}

Alice: 1, Bob: 1
     ┌───┐   

### Exercise C: A solution

In [9]:
# overwriting the old circuit with a new 
# one, that measures one of the qubits

divider = 1

def alice_bob_circuit(a,b):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    if a==1:
        qc.h(0)
    if b==0:
        qc.ry((-math.pi/4.0)/divider,1)
    else:
        qc.ry((math.pi/4.0)/divider,1)
    qc.measure([0,1],[0,1])
    return qc


simulator = Aer.get_backend('aer_simulator')

for divider in range(1,10):
    counts = alice_bob_measurements(simulator,verbose=False)
    print(f'\nDivider: {divider}')
    w = alice_bob_scoring(counts)
    print(f'\nGame advantage: {w:.4f}')



Divider: 1

Alice: 0, Bob: 0
	GR: 81
	GG: 429
	RR: 432
	RG: 82
	win: 861, lose: 163
	advantage: 0.8408

Alice: 0, Bob: 1
	GR: 75
	RG: 87
	GG: 437
	RR: 425
	win: 862, lose: 162
	advantage: 0.8418

Alice: 1, Bob: 0
	GR: 68
	RG: 61
	GG: 449
	RR: 446
	win: 895, lose: 129
	advantage: 0.8740

Alice: 1, Bob: 1
	GG: 64
	RR: 81
	RG: 473
	GR: 406
	win: 879, lose: 145
	advantage: 0.8584

Game advantage: 0.8538

Divider: 2

Alice: 0, Bob: 0
	GR: 22
	RG: 24
	RR: 482
	GG: 496
	win: 978, lose: 46
	advantage: 0.9551

Alice: 0, Bob: 1
	GR: 17
	RG: 21
	RR: 514
	GG: 472
	win: 986, lose: 38
	advantage: 0.9629

Alice: 1, Bob: 0
	RG: 179
	GG: 345
	RR: 358
	GR: 142
	win: 703, lose: 321
	advantage: 0.6865

Alice: 1, Bob: 1
	GR: 357
	RR: 158
	GG: 168
	RG: 341
	win: 698, lose: 326
	advantage: 0.6816

Game advantage: 0.8215

Divider: 3

Alice: 0, Bob: 0
	GR: 11
	RG: 10
	GG: 485
	RR: 518
	win: 1003, lose: 21
	advantage: 0.9795

Alice: 0, Bob: 1
	GR: 12
	RG: 10
	GG: 487
	RR: 515
	win: 1002, lose: 22
	advantage: 0